# Dot-Product Self-Attention Mechanism

Implementing the world known algorithm from [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf) !!


> used in transformer architecture



Self attention is needed for quantifying the importance of a word in a sentence.

Therefore, each word in a sentence is represented by a vector which depends on the other words. It creates a **context-based embedding**.

This is a Python implementation for the mathematical operations behind self-attention.

When we represent a sentence, we represent it by using the biaised representations from each word's point of view. That word is the query.

We use three weights matrices **Wq, Wk and Wv**, representing queries, keys and values. We want to find the strength of the relationship of one word to another, therefore the similarity of two vectors, one which is the query, and the other the key.

We obtain each of those vectors by multiplying the encoded word by the according matrix


> `the context vector is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key`

directly sourced from *Attention is all you need*

---



In [3]:
import torch

# example input sentence to tokenize
sentence = "The sofa does not care about board games"

# tokens
wdict={w:t for t,w in enumerate(sentence.split())}

# sentence tensor
sentence_t=torch.tensor([wdict[w] for w in sentence.split()])
sentence_t

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [5]:
#suppose we have 10 dimensional word encoding (usually in small LLMS it is at least ~~50 or 100)
em_layer= torch.nn.Embedding(len(sentence), 10)
sentence_repr = em_layer(sentence_t)
sentence_repr.shape

torch.Size([8, 10])

Each word is represented by a query vector, which is like its individual representation, and its key vector, which would be a more open representation.

This can be identified to query/key pairs in a search.

To obtain the attention vector for a given word, we compute the dot product of its query matrix with the key matrices of all the words. This gives us the weight matrix.



We can see that the key and query weight matrices need to be of the same size. But the value matrix does not.

The number of rows is equal to the dimension of the words' embedding

In [10]:
# Here we generate them randomly, in reality the weights are learned

Wq=torch.rand(10,18)
Wk=torch.rand(10, 18)
Wv=torch.rand(10, 22)

In [14]:
# Let's compute the attention for the first word
q1=torch.matmul(sentence_repr[0], Wq)
k1=torch.matmul(sentence_repr[0], Wk)
v1=torch.matmul(sentence_repr[0], Wv)

We compute the similarity of a query and each key

To compute vector similarity, is also possible to alter a bit the algorithm here, for example by taking the function similarity (like cosine similarity) :of the key and value matrices instead of the dot product.

In [18]:
# let's obtain the key and value vectors for the other words
keys=torch.matmul(sentence_repr, Wk)
values=torch.matmul(sentence_repr, Wv)

print(keys.shape)

# now we compute the dot product of q1 and each of the key in the weighted key matrix
q1w=torch.matmul(keys, q1)
print(q1w)

torch.Size([8, 18])
tensor([183.8672,  89.1740, -20.8962,  37.1406, 126.8375, 101.9559, -33.7133,
         51.4582], grad_fn=<MvBackward0>)


We now normalize this result with the softmax function, it is also common practice to divide beforehand the result by the square root of the key matrix dimension - here 18

In [20]:
q1w_norm=torch.nn.functional.softmax(q1w/(Wk.shape[1])**0.5, dim=0)

In [22]:
q1w_norm

tensor([1.0000e+00, 2.0268e-10, 1.0954e-21, 9.5597e-16, 1.4528e-06, 4.1230e-09,
        5.3400e-23, 2.7929e-14], grad_fn=<SoftmaxBackward0>)

Now let's calculate a weighted sum of these weights using the value associated to each word in the value matrice

In [24]:
q1_attention=torch.matmul(q1w_norm, values)
q1_attention

tensor([4.1085, 4.1209, 4.8486, 3.9491, 3.3074, 3.4146, 2.5324, 3.4020, 4.2248,
        3.2138, 1.6662, 3.9172, 2.5095, 3.4351, 2.9736, 3.9698, 3.7834, 3.2706,
        2.0679, 3.7417, 2.5776, 4.4017], grad_fn=<SqueezeBackward4>)

We now have our attention vector for the sequence at the 1st word! We can do that same process for a full matrix with rows of inputs and compute the attention at once.